# Анализ эффективности маркетплейса бразильских товаров

## Контекст проекта

Работаю аналитиком данных в стартапе, который создает новый маркетплейс по продаже эксклюзивных товаров из Бразилии. Компания недавно вышла на рынок и столкнулась с проблемой стагнации выручки в течение нескольких месяцев.

## Бизнес-проблема

Продакт-менеджер Петя обеспокоен отсутствием роста ключевых метрик маркетплейса. Основные ограничения:
- Необходимость роста метрик
- Сохранение положительного пользовательского опыта
- Отсутствие неудобств для клиентов

## Цели анализа

### Основные задачи:
1. **Оценка retention** - месячный retention в оформление заказа через когортный анализ
2. **Product/Market Fit** - определение соответствия продукта рынку
3. **Ключевые метрики** - выделение 5 основных метрик для максимизации прибыли
4. **Приоритизация гипотез** - выбор лучшей гипотезы с помощью фреймворка ICE
5. **Метрики влияния** - формулировка метрик для оценки эффективности гипотез
6. **Отчетность** - составление выводов и рекомендаций

## Данные для анализа

### Доступные датасеты:
1. **olist_customers_dataset.csv** - данные о пользователях
   - customer_id, customer_unique_id
   - Географические данные (zip code, city, state)

2. **olist_orders_dataset.csv** - информация о заказах
   - order_id, customer_id, order_status
   - Временные метки заказов и доставки

3. **olist_order_items_dataset.csv** - товарные позиции
   - order_id, product_id, seller_id
   - Цены, вес товаров, сроки доставки

### Статусы заказов:
- created, approved, invoiced, processing
- shipped, delivered, unavailable, canceled

## Методология

### Подход к решению:
1. **Когортный анализ** - оценка удержания клиентов
2. **Анализ PMF** - определение соответствия продукта рынку
3. **Метрический анализ** - выделение ключевых KPI
4. **ICE-фреймворк** - приоритизация гипотез развития
5. **Формулировка метрик** - для отслеживания эффективности изменений

Приступаю к комплексному анализу эффективности маркетплейса и разработке стратегии роста.

In [300]:
import pandas as pd
import numpy as np
from dateutil import parser
import matplotlib.pylab as plt
import seaborn as sns
from operator import attrgetter
import plotly.express as px

In [245]:
olist_customers_dataset = pd.read_csv('data/olist_customers_dataset.csv')
olist_order_items_dataset = pd.read_csv('data/olist_order_items_dataset.csv')
olist_orders_dataset = pd.read_csv('data/olist_orders_dataset.csv')

In [246]:
olist_customers_dataset

,customer_id,customer_unique_id,customer_zip_code_prefix,customer_city,customer_state
0,06b8999e2fba1a1fbc88172c00ba8bc7,861eff4711a542e4b93843c6dd7febb0,14409,franca,SP
1,18955e83d337fd6b2def6b18a428ac77,290c77bc529b7ac935b93aa66c333dc3,9790,sao bernardo do campo,SP
2,4e7b3e00288586ebd08712fdd0374a03,060e732b5b29e8181a18229c7b0b2b5e,1151,sao paulo,SP
3,b2b6027bc5c5109e529d4dc6358b12c3,259dac757896d24d7702b9acbbff3f3c,8775,mogi das cruzes,SP
4,4f2d8ab171c80ec8364f7c12e35b23ad,345ecd01c38d18a9036ed96c73b8d066,13056,campinas,SP
...,...,...,...,...,...
99436,17ddf5dd5d51696bb3d7c6291687be6f,1a29b476fee25c95fbafc67c5ac95cf8,3937,sao paulo,SP
99437,e7b71a9017aa05c9a7fd292d714858e8,d52a67c98be1cf6a5c84435bd38d095d,6764,taboao da serra,SP
99438,5e28dfe12db7fb50a4b2f691faecea5e,e9f50caf99f032f0bf3c55141f019d99,60115,fortaleza,CE
99439,56b18e2166679b8a959d72dd06da27f9,73c2643a0a458b49f58cea58833b192e,92120,canoas,RS


In [247]:
olist_customers_dataset.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 99441 entries, 0 to 99440
Data columns (total 5 columns):
 #   Column                    Non-Null Count  Dtype 
---  ------                    --------------  ----- 
 0   customer_id               99441 non-null  object
 1   customer_unique_id        99441 non-null  object
 2   customer_zip_code_prefix  99441 non-null  int64 
 3   customer_city             99441 non-null  object
 4   customer_state            99441 non-null  object
dtypes: int64(1), object(4)
memory usage: 3.8+ MB


In [248]:
olist_order_items_dataset

,order_id,order_item_id,product_id,seller_id,shipping_limit_date,price,freight_value
0,00010242fe8c5a6d1ba2dd792cb16214,1,4244733e06e7ecb4970a6e2683c13e61,48436dade18ac8b2bce089ec2a041202,2017-09-19 09:45:35,58.90,13.29
1,00018f77f2f0320c557190d7a144bdd3,1,e5f2d52b802189ee658865ca93d83a8f,dd7ddc04e1b6c2c614352b383efe2d36,2017-05-03 11:05:13,239.90,19.93
2,000229ec398224ef6ca0657da4fc703e,1,c777355d18b72b67abbeef9df44fd0fd,5b51032eddd242adc84c38acab88f23d,2018-01-18 14:48:30,199.00,17.87
3,00024acbcdf0a6daa1e931b038114c75,1,7634da152a4610f1595efa32f14722fc,9d7a1d34a5052409006425275ba1c2b4,2018-08-15 10:10:18,12.99,12.79
4,00042b26cf59d7ce69dfabb4e55b4fd9,1,ac6c3623068f30de03045865e4e10089,df560393f3a51e74553ab94004ba5c87,2017-02-13 13:57:51,199.90,18.14
...,...,...,...,...,...,...,...
112645,fffc94f6ce00a00581880bf54a75a037,1,4aa6014eceb682077f9dc4bffebc05b0,b8bc237ba3788b23da09c0f1f3a3288c,2018-05-02 04:11:01,299.99,43.41
112646,fffcd46ef2263f404302a634eb57f7eb,1,32e07fd915822b0765e448c4dd74c828,f3c38ab652836d21de61fb8314b69182,2018-07-20 04:31:48,350.00,36.53
112647,fffce4705a9662cd70adb13d4a31832d,1,72a30483855e2eafc67aee5dc2560482,c3cfdc648177fdbbbb35635a37472c53,2017-10-30 17:14:25,99.90,16.95
112648,fffe18544ffabc95dfada21779c9644f,1,9c422a519119dcad7575db5af1ba540e,2b3e4a2a3ea8e01938cabda2a3e5cc79,2017-08-21 00:04:32,55.99,8.72


In [249]:
olist_order_items_dataset.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 112650 entries, 0 to 112649
Data columns (total 7 columns):
 #   Column               Non-Null Count   Dtype  
---  ------               --------------   -----  
 0   order_id             112650 non-null  object 
 1   order_item_id        112650 non-null  int64  
 2   product_id           112650 non-null  object 
 3   seller_id            112650 non-null  object 
 4   shipping_limit_date  112650 non-null  object 
 5   price                112650 non-null  float64
 6   freight_value        112650 non-null  float64
dtypes: float64(2), int64(1), object(4)
memory usage: 6.0+ MB


In [250]:
olist_orders_dataset

,order_id,customer_id,order_status,order_purchase_timestamp,order_approved_at,order_delivered_carrier_date,order_delivered_customer_date,order_estimated_delivery_date
0,e481f51cbdc54678b7cc49136f2d6af7,9ef432eb6251297304e76186b10a928d,delivered,2017-10-02 10:56:33,2017-10-02 11:07:15,2017-10-04 19:55:00,2017-10-10 21:25:13,2017-10-18 00:00:00
1,53cdb2fc8bc7dce0b6741e2150273451,b0830fb4747a6c6d20dea0b8c802d7ef,delivered,2018-07-24 20:41:37,2018-07-26 03:24:27,2018-07-26 14:31:00,2018-08-07 15:27:45,2018-08-13 00:00:00
2,47770eb9100c2d0c44946d9cf07ec65d,41ce2a54c0b03bf3443c3d931a367089,delivered,2018-08-08 08:38:49,2018-08-08 08:55:23,2018-08-08 13:50:00,2018-08-17 18:06:29,2018-09-04 00:00:00
3,949d5b44dbf5de918fe9c16f97b45f8a,f88197465ea7920adcdbec7375364d82,delivered,2017-11-18 19:28:06,2017-11-18 19:45:59,2017-11-22 13:39:59,2017-12-02 00:28:42,2017-12-15 00:00:00
4,ad21c59c0840e6cb83a9ceb5573f8159,8ab97904e6daea8866dbdbc4fb7aad2c,delivered,2018-02-13 21:18:39,2018-02-13 22:20:29,2018-02-14 19:46:34,2018-02-16 18:17:02,2018-02-26 00:00:00
...,...,...,...,...,...,...,...,...
99436,9c5dedf39a927c1b2549525ed64a053c,39bd1228ee8140590ac3aca26f2dfe00,delivered,2017-03-09 09:54:05,2017-03-09 09:54:05,2017-03-10 11:18:03,2017-03-17 15:08:01,2017-03-28 00:00:00
99437,63943bddc261676b46f01ca7ac2f7bd8,1fca14ff2861355f6e5f14306ff977a7,delivered,2018-02-06 12:58:58,2018-02-06 13:10:37,2018-02-07 23:22:42,2018-02-28 17:37:56,2018-03-02 00:00:00
99438,83c1379a015df1e13d02aae0204711ab,1aa71eb042121263aafbe80c1b562c9c,delivered,2017-08-27 14:46:43,2017-08-27 15:04:16,2017-08-28 20:52:26,2017-09-21 11:24:17,2017-09-27 00:00:00
99439,11c177c8e97725db2631073c19f07b62,b331b74b18dc79bcdf6532d51e1637c1,delivered,2018-01-08 21:28:27,2018-01-08 21:36:21,2018-01-12 15:35:03,2018-01-25 23:32:54,2018-02-15 00:00:00


In [251]:
olist_orders_dataset.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 99441 entries, 0 to 99440
Data columns (total 8 columns):
 #   Column                         Non-Null Count  Dtype 
---  ------                         --------------  ----- 
 0   order_id                       99441 non-null  object
 1   customer_id                    99441 non-null  object
 2   order_status                   99441 non-null  object
 3   order_purchase_timestamp       99441 non-null  object
 4   order_approved_at              99281 non-null  object
 5   order_delivered_carrier_date   97658 non-null  object
 6   order_delivered_customer_date  96476 non-null  object
 7   order_estimated_delivery_date  99441 non-null  object
dtypes: object(8)
memory usage: 6.1+ MB


### Задача 1: Анализ месячного retention через когортный анализ

Для оценки возвращаемости клиентов в маркетплейс проведен когортный анализ месячного retention в оформлении заказов.

**Методология расчета:**

**Определение успешного заказа:**
Учитываются только заказы со статусом 'delivered' - успешно доставленные клиенту, что гарантирует реальное завершение транзакции.

**Процесс построения когорт:**
- Определение когорты: первый месяц совершения заказа для каждого уникального клиента
- Расчет периодов: разница в месяцах между текущим заказом и первым заказом клиента
- Группировка по когортам и периодам с подсчетом уникальных клиентов
- Расчет retention rate как доля клиентов от первоначального размера когорты



In [252]:
olist_orders_dataset.order_purchase_timestamp = pd.to_datetime(olist_orders_dataset.order_purchase_timestamp)
olist_orders_dataset.order_approved_at = pd.to_datetime(olist_orders_dataset.order_approved_at)
olist_orders_dataset.order_delivered_carrier_date = pd.to_datetime(olist_orders_dataset.order_delivered_carrier_date)
olist_orders_dataset.order_delivered_customer_date = pd.to_datetime(olist_orders_dataset.order_delivered_customer_date)
olist_orders_dataset.order_estimated_delivery_date = pd.to_datetime(olist_orders_dataset.order_estimated_delivery_date)



In [253]:
delivered_orders = olist_orders_dataset[olist_orders_dataset.order_status	== 'delivered']
delivered_orders = delivered_orders.merge(olist_customers_dataset[['customer_id', 'customer_unique_id']],how = 'left', on = 'customer_id')
delivered_orders

,order_id,customer_id,order_status,order_purchase_timestamp,order_approved_at,order_delivered_carrier_date,order_delivered_customer_date,order_estimated_delivery_date,customer_unique_id
0,e481f51cbdc54678b7cc49136f2d6af7,9ef432eb6251297304e76186b10a928d,delivered,2017-10-02 10:56:33,2017-10-02 11:07:15,2017-10-04 19:55:00,2017-10-10 21:25:13,2017-10-18,7c396fd4830fd04220f754e42b4e5bff
1,53cdb2fc8bc7dce0b6741e2150273451,b0830fb4747a6c6d20dea0b8c802d7ef,delivered,2018-07-24 20:41:37,2018-07-26 03:24:27,2018-07-26 14:31:00,2018-08-07 15:27:45,2018-08-13,af07308b275d755c9edb36a90c618231
2,47770eb9100c2d0c44946d9cf07ec65d,41ce2a54c0b03bf3443c3d931a367089,delivered,2018-08-08 08:38:49,2018-08-08 08:55:23,2018-08-08 13:50:00,2018-08-17 18:06:29,2018-09-04,3a653a41f6f9fc3d2a113cf8398680e8
3,949d5b44dbf5de918fe9c16f97b45f8a,f88197465ea7920adcdbec7375364d82,delivered,2017-11-18 19:28:06,2017-11-18 19:45:59,2017-11-22 13:39:59,2017-12-02 00:28:42,2017-12-15,7c142cf63193a1473d2e66489a9ae977
4,ad21c59c0840e6cb83a9ceb5573f8159,8ab97904e6daea8866dbdbc4fb7aad2c,delivered,2018-02-13 21:18:39,2018-02-13 22:20:29,2018-02-14 19:46:34,2018-02-16 18:17:02,2018-02-26,72632f0f9dd73dfee390c9b22eb56dd6
...,...,...,...,...,...,...,...,...,...
96473,9c5dedf39a927c1b2549525ed64a053c,39bd1228ee8140590ac3aca26f2dfe00,delivered,2017-03-09 09:54:05,2017-03-09 09:54:05,2017-03-10 11:18:03,2017-03-17 15:08:01,2017-03-28,6359f309b166b0196dbf7ad2ac62bb5a
96474,63943bddc261676b46f01ca7ac2f7bd8,1fca14ff2861355f6e5f14306ff977a7,delivered,2018-02-06 12:58:58,2018-02-06 13:10:37,2018-02-07 23:22:42,2018-02-28 17:37:56,2018-03-02,da62f9e57a76d978d02ab5362c509660
96475,83c1379a015df1e13d02aae0204711ab,1aa71eb042121263aafbe80c1b562c9c,delivered,2017-08-27 14:46:43,2017-08-27 15:04:16,2017-08-28 20:52:26,2017-09-21 11:24:17,2017-09-27,737520a9aad80b3fbbdad19b66b37b30
96476,11c177c8e97725db2631073c19f07b62,b331b74b18dc79bcdf6532d51e1637c1,delivered,2018-01-08 21:28:27,2018-01-08 21:36:21,2018-01-12 15:35:03,2018-01-25 23:32:54,2018-02-15,5097a5312c8b157bb7be58ae360ef43c


In [254]:
data_copy = delivered_orders.copy()
data_copy

,order_id,customer_id,order_status,order_purchase_timestamp,order_approved_at,order_delivered_carrier_date,order_delivered_customer_date,order_estimated_delivery_date,customer_unique_id
0,e481f51cbdc54678b7cc49136f2d6af7,9ef432eb6251297304e76186b10a928d,delivered,2017-10-02 10:56:33,2017-10-02 11:07:15,2017-10-04 19:55:00,2017-10-10 21:25:13,2017-10-18,7c396fd4830fd04220f754e42b4e5bff
1,53cdb2fc8bc7dce0b6741e2150273451,b0830fb4747a6c6d20dea0b8c802d7ef,delivered,2018-07-24 20:41:37,2018-07-26 03:24:27,2018-07-26 14:31:00,2018-08-07 15:27:45,2018-08-13,af07308b275d755c9edb36a90c618231
2,47770eb9100c2d0c44946d9cf07ec65d,41ce2a54c0b03bf3443c3d931a367089,delivered,2018-08-08 08:38:49,2018-08-08 08:55:23,2018-08-08 13:50:00,2018-08-17 18:06:29,2018-09-04,3a653a41f6f9fc3d2a113cf8398680e8
3,949d5b44dbf5de918fe9c16f97b45f8a,f88197465ea7920adcdbec7375364d82,delivered,2017-11-18 19:28:06,2017-11-18 19:45:59,2017-11-22 13:39:59,2017-12-02 00:28:42,2017-12-15,7c142cf63193a1473d2e66489a9ae977
4,ad21c59c0840e6cb83a9ceb5573f8159,8ab97904e6daea8866dbdbc4fb7aad2c,delivered,2018-02-13 21:18:39,2018-02-13 22:20:29,2018-02-14 19:46:34,2018-02-16 18:17:02,2018-02-26,72632f0f9dd73dfee390c9b22eb56dd6
...,...,...,...,...,...,...,...,...,...
96473,9c5dedf39a927c1b2549525ed64a053c,39bd1228ee8140590ac3aca26f2dfe00,delivered,2017-03-09 09:54:05,2017-03-09 09:54:05,2017-03-10 11:18:03,2017-03-17 15:08:01,2017-03-28,6359f309b166b0196dbf7ad2ac62bb5a
96474,63943bddc261676b46f01ca7ac2f7bd8,1fca14ff2861355f6e5f14306ff977a7,delivered,2018-02-06 12:58:58,2018-02-06 13:10:37,2018-02-07 23:22:42,2018-02-28 17:37:56,2018-03-02,da62f9e57a76d978d02ab5362c509660
96475,83c1379a015df1e13d02aae0204711ab,1aa71eb042121263aafbe80c1b562c9c,delivered,2017-08-27 14:46:43,2017-08-27 15:04:16,2017-08-28 20:52:26,2017-09-21 11:24:17,2017-09-27,737520a9aad80b3fbbdad19b66b37b30
96476,11c177c8e97725db2631073c19f07b62,b331b74b18dc79bcdf6532d51e1637c1,delivered,2018-01-08 21:28:27,2018-01-08 21:36:21,2018-01-12 15:35:03,2018-01-25 23:32:54,2018-02-15,5097a5312c8b157bb7be58ae360ef43c


In [255]:
data_copy['order_period_day'] = data_copy['order_purchase_timestamp'].dt.to_period('D')
data_copy['cohort'] = data_copy.groupby('customer_unique_id')['order_purchase_timestamp'].transform('min').dt.to_period('M')
data_copy['cohort_day'] = data_copy.groupby('customer_unique_id')['order_purchase_timestamp'].transform('min').dt.to_period('D')
data_copy

,order_id,customer_id,order_status,order_purchase_timestamp,order_approved_at,order_delivered_carrier_date,order_delivered_customer_date,order_estimated_delivery_date,customer_unique_id,order_period_day,cohort,cohort_day
0,e481f51cbdc54678b7cc49136f2d6af7,9ef432eb6251297304e76186b10a928d,delivered,2017-10-02 10:56:33,2017-10-02 11:07:15,2017-10-04 19:55:00,2017-10-10 21:25:13,2017-10-18,7c396fd4830fd04220f754e42b4e5bff,2017-10-02,2017-09,2017-09-04
1,53cdb2fc8bc7dce0b6741e2150273451,b0830fb4747a6c6d20dea0b8c802d7ef,delivered,2018-07-24 20:41:37,2018-07-26 03:24:27,2018-07-26 14:31:00,2018-08-07 15:27:45,2018-08-13,af07308b275d755c9edb36a90c618231,2018-07-24,2018-07,2018-07-24
2,47770eb9100c2d0c44946d9cf07ec65d,41ce2a54c0b03bf3443c3d931a367089,delivered,2018-08-08 08:38:49,2018-08-08 08:55:23,2018-08-08 13:50:00,2018-08-17 18:06:29,2018-09-04,3a653a41f6f9fc3d2a113cf8398680e8,2018-08-08,2018-08,2018-08-08
3,949d5b44dbf5de918fe9c16f97b45f8a,f88197465ea7920adcdbec7375364d82,delivered,2017-11-18 19:28:06,2017-11-18 19:45:59,2017-11-22 13:39:59,2017-12-02 00:28:42,2017-12-15,7c142cf63193a1473d2e66489a9ae977,2017-11-18,2017-11,2017-11-18
4,ad21c59c0840e6cb83a9ceb5573f8159,8ab97904e6daea8866dbdbc4fb7aad2c,delivered,2018-02-13 21:18:39,2018-02-13 22:20:29,2018-02-14 19:46:34,2018-02-16 18:17:02,2018-02-26,72632f0f9dd73dfee390c9b22eb56dd6,2018-02-13,2018-02,2018-02-13
...,...,...,...,...,...,...,...,...,...,...,...,...
96473,9c5dedf39a927c1b2549525ed64a053c,39bd1228ee8140590ac3aca26f2dfe00,delivered,2017-03-09 09:54:05,2017-03-09 09:54:05,2017-03-10 11:18:03,2017-03-17 15:08:01,2017-03-28,6359f309b166b0196dbf7ad2ac62bb5a,2017-03-09,2017-03,2017-03-09
96474,63943bddc261676b46f01ca7ac2f7bd8,1fca14ff2861355f6e5f14306ff977a7,delivered,2018-02-06 12:58:58,2018-02-06 13:10:37,2018-02-07 23:22:42,2018-02-28 17:37:56,2018-03-02,da62f9e57a76d978d02ab5362c509660,2018-02-06,2018-02,2018-02-06
96475,83c1379a015df1e13d02aae0204711ab,1aa71eb042121263aafbe80c1b562c9c,delivered,2017-08-27 14:46:43,2017-08-27 15:04:16,2017-08-28 20:52:26,2017-09-21 11:24:17,2017-09-27,737520a9aad80b3fbbdad19b66b37b30,2017-08-27,2017-08,2017-08-27
96476,11c177c8e97725db2631073c19f07b62,b331b74b18dc79bcdf6532d51e1637c1,delivered,2018-01-08 21:28:27,2018-01-08 21:36:21,2018-01-12 15:35:03,2018-01-25 23:32:54,2018-02-15,5097a5312c8b157bb7be58ae360ef43c,2018-01-08,2018-01,2018-01-08


In [256]:
data_copy['period_number_month'] = np.floor((data_copy.order_period_day - data_copy.cohort_day).apply(attrgetter('n')) / 30)
data_copy

,order_id,customer_id,order_status,order_purchase_timestamp,order_approved_at,order_delivered_carrier_date,order_delivered_customer_date,order_estimated_delivery_date,customer_unique_id,order_period_day,cohort,cohort_day,period_number_month
0,e481f51cbdc54678b7cc49136f2d6af7,9ef432eb6251297304e76186b10a928d,delivered,2017-10-02 10:56:33,2017-10-02 11:07:15,2017-10-04 19:55:00,2017-10-10 21:25:13,2017-10-18,7c396fd4830fd04220f754e42b4e5bff,2017-10-02,2017-09,2017-09-04,0.0
1,53cdb2fc8bc7dce0b6741e2150273451,b0830fb4747a6c6d20dea0b8c802d7ef,delivered,2018-07-24 20:41:37,2018-07-26 03:24:27,2018-07-26 14:31:00,2018-08-07 15:27:45,2018-08-13,af07308b275d755c9edb36a90c618231,2018-07-24,2018-07,2018-07-24,0.0
2,47770eb9100c2d0c44946d9cf07ec65d,41ce2a54c0b03bf3443c3d931a367089,delivered,2018-08-08 08:38:49,2018-08-08 08:55:23,2018-08-08 13:50:00,2018-08-17 18:06:29,2018-09-04,3a653a41f6f9fc3d2a113cf8398680e8,2018-08-08,2018-08,2018-08-08,0.0
3,949d5b44dbf5de918fe9c16f97b45f8a,f88197465ea7920adcdbec7375364d82,delivered,2017-11-18 19:28:06,2017-11-18 19:45:59,2017-11-22 13:39:59,2017-12-02 00:28:42,2017-12-15,7c142cf63193a1473d2e66489a9ae977,2017-11-18,2017-11,2017-11-18,0.0
4,ad21c59c0840e6cb83a9ceb5573f8159,8ab97904e6daea8866dbdbc4fb7aad2c,delivered,2018-02-13 21:18:39,2018-02-13 22:20:29,2018-02-14 19:46:34,2018-02-16 18:17:02,2018-02-26,72632f0f9dd73dfee390c9b22eb56dd6,2018-02-13,2018-02,2018-02-13,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...
96473,9c5dedf39a927c1b2549525ed64a053c,39bd1228ee8140590ac3aca26f2dfe00,delivered,2017-03-09 09:54:05,2017-03-09 09:54:05,2017-03-10 11:18:03,2017-03-17 15:08:01,2017-03-28,6359f309b166b0196dbf7ad2ac62bb5a,2017-03-09,2017-03,2017-03-09,0.0
96474,63943bddc261676b46f01ca7ac2f7bd8,1fca14ff2861355f6e5f14306ff977a7,delivered,2018-02-06 12:58:58,2018-02-06 13:10:37,2018-02-07 23:22:42,2018-02-28 17:37:56,2018-03-02,da62f9e57a76d978d02ab5362c509660,2018-02-06,2018-02,2018-02-06,0.0
96475,83c1379a015df1e13d02aae0204711ab,1aa71eb042121263aafbe80c1b562c9c,delivered,2017-08-27 14:46:43,2017-08-27 15:04:16,2017-08-28 20:52:26,2017-09-21 11:24:17,2017-09-27,737520a9aad80b3fbbdad19b66b37b30,2017-08-27,2017-08,2017-08-27,0.0
96476,11c177c8e97725db2631073c19f07b62,b331b74b18dc79bcdf6532d51e1637c1,delivered,2018-01-08 21:28:27,2018-01-08 21:36:21,2018-01-12 15:35:03,2018-01-25 23:32:54,2018-02-15,5097a5312c8b157bb7be58ae360ef43c,2018-01-08,2018-01,2018-01-08,0.0


In [257]:
df_cohort = data_copy.groupby(['cohort', 'period_number_month']).agg(n_customers=('customer_unique_id', 'nunique')).reset_index()

In [258]:
cohort_pivot = df_cohort.pivot(index = 'cohort',columns='period_number_month',values = 'n_customers')

In [262]:
cohort_size = cohort_pivot.iloc[:, 0]
retention_mtrx = cohort_pivot.divide(cohort_size,axis=0)

In [263]:
retention_mtrx

period_number_month,0.0,1.0,2.0,3.0,4.0,5.0,6.0,7.0,8.0,9.0,...,11.0,12.0,13.0,14.0,15.0,16.0,17.0,19.0,20.0,21.0
cohort,,,,,,,,,,,,,,,,,,,,,
2016-09,1.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2016-10,1.0,NaN,NaN,NaN,NaN,NaN,0.003817,NaN,NaN,0.003817,...,0.003817,NaN,0.003817,NaN,0.003817,NaN,0.003817,0.007634,0.003817,0.003817
2016-12,1.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2017-01,1.0,0.002789,NaN,0.005579,0.001395,0.001395,0.002789,0.002789,NaN,NaN,...,0.005579,0.004184,0.002789,NaN,0.002789,0.002789,0.002789,0.001395,NaN,NaN
2017-02,1.0,0.001229,0.001843,0.003686,0.002457,0.001229,0.002457,0.000614,0.001229,0.002457,...,0.003686,0.000614,0.001843,0.001229,0.000614,0.000614,0.001843,NaN,NaN,NaN
2017-03,1.0,0.002797,0.005194,0.004395,0.001199,0.000799,0.003596,0.001598,0.002397,0.002397,...,0.001598,0.001598,0.001199,0.002797,0.000799,0.001199,0.000799,NaN,NaN,NaN
2017-04,1.0,0.003546,0.002660,0.000443,0.002660,0.003546,0.002660,0.004433,0.002216,0.002660,...,0.001330,NaN,0.000887,0.000887,0.000887,0.000887,NaN,NaN,NaN,NaN
2017-05,1.0,0.004057,0.004057,0.002318,0.003767,0.003187,0.003187,0.001159,0.002898,0.002608,...,0.003187,0.001739,0.000869,0.002028,0.001159,NaN,NaN,NaN,NaN,NaN
2017-06,1.0,0.004281,0.004939,0.003622,0.001646,0.005268,0.002963,0.001976,0.001317,0.003293,...,0.002963,0.000988,0.002963,0.001317,NaN,NaN,NaN,NaN,NaN,NaN


In [265]:
average_retention_first_month = retention_mtrx[1].median()
average_retention_first_month

0.0035460992907801418

In [266]:
highest_retention_third_month = retention_matrix[3].idxmax()
highest_retention_value = retention_matrix[3].max()

print(f'Когорта с самым высоким retention на третий месяц: {highest_retention_third_month}, значение: {highest_retention_value:.4f}%')

Когорта с самым высоким retention на третий месяц: 2017-02, значение: 0.0059%


**Ключевые результаты:**

**Медианный retention 1-го месяца:** 0.35% 
- Показывает очень низкую долю клиентов, возвращающихся через месяц после первой покупки

**Лучшая когорта по retention на 3-й месяц:** 2017-02
- Retention значение: 0.0059%
- Даже лучшая когорта демонстрирует крайне низкие показатели удержания

**Вывод:**
Retention rates находятся на критически низком уровне, что указывает на серьезные проблемы с удержанием клиентов и необходимость срочных мер по улучшению пользовательского опыта и лояльности.

### Задача 2: Оценка Product/Market Fit маркетплейса

На основе проведенного когортного анализа определено соответствие продукта рынку (Product/Market Fit).

**Оценка PMF:**

**❌ Product/Market Fit отсутствует**

1. Привлечение новых клиентов: Когортный анализ показывает, что каждый месяц в продукт приходит значительное количество новых клиентов, с медианным значением в 4057 клиентов. Этот стабильный поток новых пользователей указывает на то, что маркетплейс способен привлекать интересующихся клиентов, которые готовы совершать покупки. Однако важно отметить, что наличие стабильного потока новых клиентов не всегда гарантирует долгосрочный успех бизнеса.

2. Проблемы удержания клиентов: Несмотря на привлекательность для новых пользователей, когортный анализ показывает, что лишь 0.3% клиентов (по медиане) остаются активными на первый месяц после первой покупки, а на второй месяц этот показатель значительно снижается до нуля. Это говорит о том, что большинство новых пользователей не находят достаточной ценности в продукте или услуге, чтобы продолжать использовать его. Высокий уровень оттока на ранних этапах взаимодействия с продуктом указывает на потенциальные проблемы с его качеством, функциональностью или соответствием ожиданиям клиентов.

3. Необходимость в улучшении ценностного предложения: Низкий уровень удержания клиентов может свидетельствовать о том, что маркетплейс не предоставляет достаточной ценности или уникальности по сравнению с конкурентами. Это может быть связано с недостаточной дифференциацией продукта, проблемами с качеством обслуживания или отсутствием необходимых функций и услуг.

В целом, текущие данные о retention указывают на слабый product/market fit для данного маркетплейса. Стабильный поток новых клиентов является положительным знаком, однако низкий уровень удержания сигнализирует о необходимости серьезных изменений в подходе к продукту и клиентскому обслуживанию.
можешь просто вот это перефразировать

### Задача 3. Определить 5 основных метрик, на которых продакт может сконцентрироваться, чтобы максимизировать прибыль компании.

1. **Рост продаж маркетплейса**

Метрика: Общий объем продаж (Gross Merchandise Volume, GMV)

Описание: Эта метрика отражает общий объем продаж через маркетплейс за определенный период. Она показывает, насколько успешно маркетплейс привлекает покупателей и генерирует доход.

In [306]:
delivered_orders_price = delivered_orders.merge(olist_order_items_dataset[['order_id','price']],on = 'order_id')
delivered_orders_price['delivered_m'] = delivered_orders.order_purchase_timestamp.dt.to_period('M')
delivered_orders_price = delivered_orders_price[delivered_orders_price.delivered_m<='2018-08']
m_gvm = delivered_orders_price.groupby('delivered_m').agg(GVM = ('price','sum')).reset_index()
m_gvm.delivered_m = m_gvm.delivered_m.astype(str)
m_gvm = m_gvm.rename(columns={'delivered_m':'mounth'})
m_gvm

,mounth,GVM
0,2016-09,135.00
1,2016-10,30389.70
2,2016-12,114.00
3,2017-01,90059.45
4,2017-02,196754.50
5,2017-03,298380.13
6,2017-04,262397.48
7,2017-05,441200.13
8,2017-06,361768.62
9,2017-07,466169.23


In [349]:
plt.figure(figsize=(10, 6))
px.line(m_gvm,x='mounth',y = 'GVM',title='Marketplace Sales Growth (GMV)',markers='o')


<Figure size 1000x600 with 0 Axes>

2. **Объем аудитории, которой продукт доставляет ценность**

Метрика: Количество платящих клиентов (Paying Users)

Описание: Эта метрика показывает количество уникальных пользователей, которые совершают хотя бы один заказ в продукте в течение месяца. Она помогает оценить, сколько клиентов на самом деле извлекают ценность из маркетплейса, то есть находят тот товар, который им нужен.

In [329]:
paying_users = delivered_orders_price.groupby('delivered_m',as_index=False).agg(Number_of_paying_customers = ('customer_unique_id','nunique'))
paying_users = paying_users.rename(columns={'delivered_m':'mounth'})
paying_users.mounth = paying_users.mounth.astype(str)
paying_users

,mounth,Number_of_paying_customers
0,2016-09,1
1,2016-10,265
2,2016-12,1
3,2017-01,750
4,2017-02,1646
5,2017-03,2531
6,2017-04,2288
7,2017-05,3522
8,2017-06,3114
9,2017-07,3832


In [348]:
plt.figure(figsize=(10, 6))
px.line(paying_users,x='mounth',y = 'Number_of_paying_customers',title='Number of paying customers (Paying Users)',markers='o')

<Figure size 1000x600 with 0 Axes>

3. **Заинтересованность новых клиентов в продукте**

Метрика: Конверсия в первую покупку (Conversion to Paying User)

Описание: Эта метрика измеряет процент новых пользователей, которые совершают покупку после первого визита на сайт или в приложение. Она отражает, насколько эффективно маркетплейс привлекает и конвертирует новых клиентов.


(К сожалению, данную метрику невозможно посчитать по имеющимся данным, но я бы хотел отметить ее, так как она важна для оценки заинтересованности клиентов)

4. **Вовлеченность клиента в продолжение использования продукта**

Метрика: Коэффициент удержания (Retention Rate)

Описание: Эта метрика показывает процент клиентов, которые продолжают использовать продукт после первого месяца. Она помогает понять, насколько хорошо продукт удерживает клиентов и удовлетворяет их потребности.

In [332]:
retention_mtrx

period_number_month,0.0,1.0,2.0,3.0,4.0,5.0,6.0,7.0,8.0,9.0,...,11.0,12.0,13.0,14.0,15.0,16.0,17.0,19.0,20.0,21.0
cohort,,,,,,,,,,,,,,,,,,,,,
2016-09,1.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2016-10,1.0,NaN,NaN,NaN,NaN,NaN,0.003817,NaN,NaN,0.003817,...,0.003817,NaN,0.003817,NaN,0.003817,NaN,0.003817,0.007634,0.003817,0.003817
2016-12,1.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2017-01,1.0,0.002789,NaN,0.005579,0.001395,0.001395,0.002789,0.002789,NaN,NaN,...,0.005579,0.004184,0.002789,NaN,0.002789,0.002789,0.002789,0.001395,NaN,NaN
2017-02,1.0,0.001229,0.001843,0.003686,0.002457,0.001229,0.002457,0.000614,0.001229,0.002457,...,0.003686,0.000614,0.001843,0.001229,0.000614,0.000614,0.001843,NaN,NaN,NaN
2017-03,1.0,0.002797,0.005194,0.004395,0.001199,0.000799,0.003596,0.001598,0.002397,0.002397,...,0.001598,0.001598,0.001199,0.002797,0.000799,0.001199,0.000799,NaN,NaN,NaN
2017-04,1.0,0.003546,0.002660,0.000443,0.002660,0.003546,0.002660,0.004433,0.002216,0.002660,...,0.001330,NaN,0.000887,0.000887,0.000887,0.000887,NaN,NaN,NaN,NaN
2017-05,1.0,0.004057,0.004057,0.002318,0.003767,0.003187,0.003187,0.001159,0.002898,0.002608,...,0.003187,0.001739,0.000869,0.002028,0.001159,NaN,NaN,NaN,NaN,NaN
2017-06,1.0,0.004281,0.004939,0.003622,0.001646,0.005268,0.002963,0.001976,0.001317,0.003293,...,0.002963,0.000988,0.002963,0.001317,NaN,NaN,NaN,NaN,NaN,NaN


5. **Денежное выражение вовлеченности клиента**

Метрика: ARPPU

Описание: Эта метрика отражает общую выручку, которую компания получает с одного клиента в среднем

In [342]:
delivered_orders_price

,order_id,customer_id,order_status,order_purchase_timestamp,order_approved_at,order_delivered_carrier_date,order_delivered_customer_date,order_estimated_delivery_date,customer_unique_id,delivered_m,price
0,e481f51cbdc54678b7cc49136f2d6af7,9ef432eb6251297304e76186b10a928d,delivered,2017-10-02 10:56:33,2017-10-02 11:07:15,2017-10-04 19:55:00,2017-10-10 21:25:13,2017-10-18,7c396fd4830fd04220f754e42b4e5bff,2017-10,29.99
1,53cdb2fc8bc7dce0b6741e2150273451,b0830fb4747a6c6d20dea0b8c802d7ef,delivered,2018-07-24 20:41:37,2018-07-26 03:24:27,2018-07-26 14:31:00,2018-08-07 15:27:45,2018-08-13,af07308b275d755c9edb36a90c618231,2018-07,118.70
2,47770eb9100c2d0c44946d9cf07ec65d,41ce2a54c0b03bf3443c3d931a367089,delivered,2018-08-08 08:38:49,2018-08-08 08:55:23,2018-08-08 13:50:00,2018-08-17 18:06:29,2018-09-04,3a653a41f6f9fc3d2a113cf8398680e8,2018-08,159.90
3,949d5b44dbf5de918fe9c16f97b45f8a,f88197465ea7920adcdbec7375364d82,delivered,2017-11-18 19:28:06,2017-11-18 19:45:59,2017-11-22 13:39:59,2017-12-02 00:28:42,2017-12-15,7c142cf63193a1473d2e66489a9ae977,2017-11,45.00
4,ad21c59c0840e6cb83a9ceb5573f8159,8ab97904e6daea8866dbdbc4fb7aad2c,delivered,2018-02-13 21:18:39,2018-02-13 22:20:29,2018-02-14 19:46:34,2018-02-16 18:17:02,2018-02-26,72632f0f9dd73dfee390c9b22eb56dd6,2018-02,19.90
...,...,...,...,...,...,...,...,...,...,...,...
96473,d8e06a0d47d09dae630bf252bbf904b9,980ce384cf75d4d521939629585ff283,delivered,2017-10-24 13:18:46,2017-10-26 02:56:25,2017-10-27 15:07:56,2017-11-01 19:49:49,2017-11-06,1487f8c57890183ff94a0c3905ac7edb,2017-03,21.99
96474,27d100303d20caf196f550e504ba6240,c9785a6ba20cc13ed3c0ac679ced9067,delivered,2017-03-18 00:23:02,2017-03-18 00:23:02,2017-03-20 15:36:45,2017-03-23 18:00:41,2017-04-10,726c8f1821bccdbb500536a36b6804bc,2018-02,69.00
96475,5d636e00c8235b39e3149dd8ddc949b8,c1b3478907a0889328e147a2a3d7ccfd,delivered,2017-08-08 20:01:23,2017-08-10 03:24:17,2017-08-11 17:23:15,2017-08-18 18:43:41,2017-09-01,f2ff32c758b416748ba01b03119594d7,2017-08,67.90
96476,684eb867dfdc67690e5160c9e1bca789,3614b14d84a7dff8478e262ed746837b,delivered,2018-04-30 01:08:25,2018-04-30 01:34:37,2018-05-03 11:09:00,2018-05-16 16:22:31,2018-05-28,591f835a0f1e740b3d75ad7b0ffdffaa,2018-01,64.99


In [345]:
pre_arppu = delivered_orders_price.groupby(['delivered_m'],as_index=False).agg(total_price = ('price','sum'),users_count = ('customer_unique_id','nunique'))
pre_arppu['ARPPU'] = pre_arppu.total_price/pre_arppu.users_count
pre_arppu = pre_arppu.rename(columns={'delivered_m':'mounth'})
pre_arppu.mounth = pre_arppu.mounth.astype(str)
pre_arppu


,mounth,total_price,users_count,ARPPU
0,2016-09,135.00,1,135.000000
1,2016-10,30389.70,265,114.678113
2,2016-12,114.00,1,114.000000
3,2017-01,90059.45,750,120.079267
4,2017-02,196754.50,1646,119.534933
5,2017-03,298380.13,2531,117.890213
6,2017-04,262397.48,2288,114.684213
7,2017-05,441200.13,3522,125.269770
8,2017-06,361768.62,3114,116.174894
9,2017-07,466169.23,3832,121.651678


In [347]:
plt.figure(figsize=(10, 6))
px.line(data_frame=pre_arppu,x='mounth',y='ARPPU',title='ARPPU',markers='o')

<Figure size 1000x600 with 0 Axes>

### Задача 4: Приоритизация гипотез с помощью фреймворка ICE

Для выбора наиболее эффективной гипотезы улучшения маркетплейса применяется фреймворк ICE (Impact, Confidence, Ease).

**Методология расчета:**

**Impact расчет:**
- Период анализа: с июня 2017 года
- Конверсия в повторный заказ: 0.35% (медианный retention 1-го месяца)

**Гипотезы для оценки:**

**Гипотеза 1:** Исправление бага в системе процессинга заказов
- Устранение всех отмен заказов
- Confidence: 8, Ease: 6

**Гипотеза 2:** Сокращение времени до отгрузки заказа  
- Улучшение сроков доставки для увеличения повторных заказов
- Confidence: 10, Ease: 4

**Гипотеза 3:** Создание нового способа оплаты
- Упрощение процесса оформления заказа
- Конвертация клиентов в повторные заказы
- Confidence: 5, Ease: 9

**Процесс расчета:**
Для каждой гипотезы будет рассчитан Impact на основе потенциального увеличения количества доставленных заказов, после чего определен ICE score = Impact × Confidence × Ease.

Приступаю к расчету Impact и определению лучшей гипотезы для реализации.

**Гипотеза 1. Если исправим баг в системе процессинга заказов, то клиентам не придется сталкиваться с проблемой отмены заказа, вследствие чего количество доставленных заказов увеличится.**

Для расчета Impact по этой гипотезе можно посчитать количество заказов, который остались в статусе отмены. Заказом в статусе отмены будем считать статусы: canceled, unavailable

In [358]:
number_of_canceled_orders = \
olist_orders_dataset[(pd.to_datetime(olist_orders_dataset.order_purchase_timestamp) >= '2017-06') & \
                    (olist_orders_dataset.order_status.isin(['canceled', 'unavailable']))]\
                    .shape[0]
print(f'Number of canceled orders: {number_of_canceled_orders}')

Number of canceled orders: 974


**Гипотеза 2.
Если сократим время до отгрузки заказа, то клиенты перестанут получать свой заказ с запаздыванием, вследствие чего количество заказов увеличится за счет повторных заказов.**

Для расчета Impact по этой гипотезе можно посчитать количество заказов, которые были доставлены с опозданием. Конверсию в повторный заказ возьмем равной 0.0035.

In [369]:
count_cancel = olist_orders_dataset[(pd.to_datetime(olist_orders_dataset.order_purchase_timestamp) >= '2017-06')\
    & (pd.to_datetime(olist_orders_dataset.order_delivered_customer_date).dt.to_period('D')\
    > pd.to_datetime(olist_orders_dataset.order_estimated_delivery_date).dt.to_period('D'))\
        & (olist_orders_dataset.order_status == 'delivered')].order_id.nunique()

print(f'Number of delayed orders: {count_cancel}')

Number of delayed orders: 6087


In [371]:
print(f'Possible number of repeat orders: {count_cancel*0.0035}')

Possible number of repeat orders: 21.3045


**Гипотеза 3.
Если создадим новый способ оплаты, который будет конвертировать клиентов в повторный заказ, то клиенты не будут испытывать трудности при выполнении заказа, вследствие чего  количество заказов увеличится за счет повторных заказов.**

Для расчета Impact по этой гипотезе можно посчитать количество заказов, где клиент сделал только единичный заказ. Конверсию в повторный заказ возьмем равной 0.0035.

In [374]:
olist_customers_dataset

,customer_id,customer_unique_id,customer_zip_code_prefix,customer_city,customer_state
0,06b8999e2fba1a1fbc88172c00ba8bc7,861eff4711a542e4b93843c6dd7febb0,14409,franca,SP
1,18955e83d337fd6b2def6b18a428ac77,290c77bc529b7ac935b93aa66c333dc3,9790,sao bernardo do campo,SP
2,4e7b3e00288586ebd08712fdd0374a03,060e732b5b29e8181a18229c7b0b2b5e,1151,sao paulo,SP
3,b2b6027bc5c5109e529d4dc6358b12c3,259dac757896d24d7702b9acbbff3f3c,8775,mogi das cruzes,SP
4,4f2d8ab171c80ec8364f7c12e35b23ad,345ecd01c38d18a9036ed96c73b8d066,13056,campinas,SP
...,...,...,...,...,...
99436,17ddf5dd5d51696bb3d7c6291687be6f,1a29b476fee25c95fbafc67c5ac95cf8,3937,sao paulo,SP
99437,e7b71a9017aa05c9a7fd292d714858e8,d52a67c98be1cf6a5c84435bd38d095d,6764,taboao da serra,SP
99438,5e28dfe12db7fb50a4b2f691faecea5e,e9f50caf99f032f0bf3c55141f019d99,60115,fortaleza,CE
99439,56b18e2166679b8a959d72dd06da27f9,73c2643a0a458b49f58cea58833b192e,92120,canoas,RS


In [384]:
olist_orders_dataset

,order_id,customer_id,order_status,order_purchase_timestamp,order_approved_at,order_delivered_carrier_date,order_delivered_customer_date,order_estimated_delivery_date
0,e481f51cbdc54678b7cc49136f2d6af7,9ef432eb6251297304e76186b10a928d,delivered,2017-10-02 10:56:33,2017-10-02 11:07:15,2017-10-04 19:55:00,2017-10-10 21:25:13,2017-10-18
1,53cdb2fc8bc7dce0b6741e2150273451,b0830fb4747a6c6d20dea0b8c802d7ef,delivered,2018-07-24 20:41:37,2018-07-26 03:24:27,2018-07-26 14:31:00,2018-08-07 15:27:45,2018-08-13
2,47770eb9100c2d0c44946d9cf07ec65d,41ce2a54c0b03bf3443c3d931a367089,delivered,2018-08-08 08:38:49,2018-08-08 08:55:23,2018-08-08 13:50:00,2018-08-17 18:06:29,2018-09-04
3,949d5b44dbf5de918fe9c16f97b45f8a,f88197465ea7920adcdbec7375364d82,delivered,2017-11-18 19:28:06,2017-11-18 19:45:59,2017-11-22 13:39:59,2017-12-02 00:28:42,2017-12-15
4,ad21c59c0840e6cb83a9ceb5573f8159,8ab97904e6daea8866dbdbc4fb7aad2c,delivered,2018-02-13 21:18:39,2018-02-13 22:20:29,2018-02-14 19:46:34,2018-02-16 18:17:02,2018-02-26
...,...,...,...,...,...,...,...,...
99436,9c5dedf39a927c1b2549525ed64a053c,39bd1228ee8140590ac3aca26f2dfe00,delivered,2017-03-09 09:54:05,2017-03-09 09:54:05,2017-03-10 11:18:03,2017-03-17 15:08:01,2017-03-28
99437,63943bddc261676b46f01ca7ac2f7bd8,1fca14ff2861355f6e5f14306ff977a7,delivered,2018-02-06 12:58:58,2018-02-06 13:10:37,2018-02-07 23:22:42,2018-02-28 17:37:56,2018-03-02
99438,83c1379a015df1e13d02aae0204711ab,1aa71eb042121263aafbe80c1b562c9c,delivered,2017-08-27 14:46:43,2017-08-27 15:04:16,2017-08-28 20:52:26,2017-09-21 11:24:17,2017-09-27
99439,11c177c8e97725db2631073c19f07b62,b331b74b18dc79bcdf6532d51e1637c1,delivered,2018-01-08 21:28:27,2018-01-08 21:36:21,2018-01-12 15:35:03,2018-01-25 23:32:54,2018-02-15


In [ ]:
orders_users = olist_orders_dataset.merge\
    (olist_customers_dataset[['customer_id','customer_unique_id']], on = 'customer_id')
orders_users = orders_users[(pd.to_datetime(orders_users.order_purchase_timestamp)>'2017-06') \
    &(orders_users.order_status == 'delivered')]
orders_users = orders_users.groupby('customer_unique_id')\
    .agg(count_of_orders = ('order_id','nunique'))

single_order = int(orders_users.query('count_of_orders ==1').count())
print(f'number of individual orders: {single_order}')

number of individual orders: 281.95300000000003


/var/folders/gv/w8lzwcsd0_d7rgz4964mrrqr0000gn/T/ipykernel_92722/735298342.py:8: FutureWarning:

Calling int on a single element Series is deprecated and will raise a TypeError in the future. Use int(ser.iloc[0]) instead



In [390]:
print(f'Possible number of individual orders: {single_order*0.0035}')

Possible number of individual orders: 281.95300000000003


При сравнении по фреймворку ICE из 3 гипотез выигрывает гипотеза 1.

Для проверки гипотезы о том, что исправление бага в системе процессинга заказов приведет к снижению количества отмен и увеличению количества доставленных заказов, важно определить метрики, которые будут отражать эффективность изменений. Метрики можно разделить на три категории: целевые, прокси и guardrail.

**Целевые метрики**

1. Количество доставленных заказов

    Определение: Общее количество заказов, которые были успешно доставлены клиентам.
    Цель: Оценить, увеличилось ли количество успешных доставок после исправления бага. Это основная метрика, которая непосредственно отражает успех гипотезы.

**Прокси метрики**

2. Конверсия в доставку товара до покупателя

    Определение: Общее количество заказов, которые были доставлены клиентам / Общее количество заказов.
    Цель: Измерить снижение числа отмен после исправления бага. Эта метрика служит индикатором того, насколько эффективно работает система после внесения изменений.

**Guardrail метрики**

3. Конверсия в оформление заказа

    Определение: Доля клиентов, которые оформили заказ, деленная на количество активной аудитории
    Цель: Убедиться, что исправление бага не негативно сказалось на общем восприятии клиента совершить следующий заказ.
